# 公式拟合

某医院进行临床登记实验, 收集白内障病例2000眼, 进行Phaco+IOL植入术, 使用SRK-2公式进行IOL计算, 

术前检查记录: 
* 选用IOL的A常数: A
* 角膜曲率: K1, K2
* 眼轴长: L
* 目标屈光度: REF

术中记录: 
* 植入IOL度数: P

术后3个月复查: 
* 仅记录了术后对屈光状态不满意, 给予眼镜处方患者的显然验光结果: PostREF
* 一部分病人失访, 术后验光结果空缺

现该医院委托你拟合一个新的IOL计算公式, 所有的病例数据已经整理好存成excel文件, 放在了data文件夹内的sampleIOLdataset.xlsx文件中. 


# 读取数据集

# 数据清洗

# 神经网络简介

# Keras简介

# 建立深度神经网络

# 训练拟合

# 使用深度神经网络进行预测